In [1]:
import pandas as pd
from datetime import datetime
import pm4py
import numpy as np
from collections import defaultdict
from Levenshtein import distance

from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn import metrics

import copy
import matplotlib as mlp
import matplotlib.pyplot as plt

In [2]:
Order = pm4py.read.read_xes("1_Order.xes")
Loops = pm4py.read.read_xes("2_Loops.xes") 
Skips = pm4py.read.read_xes("3_Skips.xes")
Order_Loops = pm4py.read.read_xes("4_Order_Loops.xes")
Loops_Skips = pm4py.read.read_xes("5_Loops_Skips.xes")
Skips_Order = pm4py.read.read_xes("6_Skips_Order.xes")
Order_Loops_Skips = pm4py.read.read_xes("7_Order_Loops_Skips.xes")


C:\Users\la1949\Anaconda3\envs\pm4py\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 18/18 [00:00<?, ?it/s]


In [3]:
Order_log = Order.groupby(['case:concept:name'])['concept:name'].apply(list).reset_index()
Loops_log = Loops.groupby(['case:concept:name'])['concept:name'].apply(list).reset_index()
Skips_log = Skips.groupby(['case:concept:name'])['concept:name'].apply(list).reset_index()
Order_Loops_log = Order_Loops.groupby(['case:concept:name'])['concept:name'].apply(list).reset_index()
Loops_Skips_log = Loops_Skips.groupby(['case:concept:name'])['concept:name'].apply(list).reset_index()
Skips_Order_log = Skips_Order.groupby(['case:concept:name'])['concept:name'].apply(list).reset_index()
Order_Loops_Skips_log = Order_Loops_Skips.groupby(['case:concept:name'])['concept:name'].apply(list).reset_index()

In [4]:
logs = [Order_log, Loops_log, Skips_log, Order_Loops_log, Loops_Skips_log, Skips_Order_log, Order_Loops_Skips_log]

In [5]:
for i in logs:
    print(i)

  case:concept:name     concept:name
0                 1  [A, B, C, D, E]
1                 2  [A, B, D, C, E]
2                 3  [A, C, B, D, E]
3                 4  [A, D, B, C, E]
4                 5  [A, C, D, B, E]
5                 6  [A, D, C, B, E]
  case:concept:name              concept:name
0                 1        [A, B, B, C, D, E]
1                 2        [A, B, C, C, D, E]
2                 3        [A, B, C, D, D, E]
3                 4     [A, B, C, B, C, D, E]
4                 5     [A, B, C, D, C, D, E]
5                 6  [A, B, C, D, B, C, D, E]
  case:concept:name  concept:name
0                 1  [A, C, D, E]
1                 2  [A, B, D, E]
2                 3  [A, B, C, E]
3                 4     [A, D, E]
4                 5     [A, B, E]
5                 6        [A, E]
   case:concept:name              concept:name
0                  1           [A, B, C, D, E]
1                 10     [A, B, C, B, C, D, E]
2                 11     [A, B, C, D, C,

In [6]:
#Change to string format
for i in logs:
    i["strings"] = i["concept:name"].apply(lambda x: ''.join(x))

In [7]:
#Calculate distance matrix based on Levenshtein Distance
from Levenshtein import distance

def Lev(logVar):
    
    List = logVar["strings"]

    n = len(List)
    dist_matrix = np.zeros((n,n))    # initialize distance matrix to a square of zeros

    for i in range(n):
        if i % 200 == 0:
            print(i)
        for j in range(i, n):
            dist_matrix[i,j] = distance(List[i], List[j])
            dist_matrix[j,i] = dist_matrix[i,j]       # for the symmetric part, no computation

    return dist_matrix

In [8]:
logs

[  case:concept:name     concept:name strings
 0                 1  [A, B, C, D, E]   ABCDE
 1                 2  [A, B, D, C, E]   ABDCE
 2                 3  [A, C, B, D, E]   ACBDE
 3                 4  [A, D, B, C, E]   ADBCE
 4                 5  [A, C, D, B, E]   ACDBE
 5                 6  [A, D, C, B, E]   ADCBE,
   case:concept:name              concept:name   strings
 0                 1        [A, B, B, C, D, E]    ABBCDE
 1                 2        [A, B, C, C, D, E]    ABCCDE
 2                 3        [A, B, C, D, D, E]    ABCDDE
 3                 4     [A, B, C, B, C, D, E]   ABCBCDE
 4                 5     [A, B, C, D, C, D, E]   ABCDCDE
 5                 6  [A, B, C, D, B, C, D, E]  ABCDBCDE,
   case:concept:name  concept:name strings
 0                 1  [A, C, D, E]    ACDE
 1                 2  [A, B, D, E]    ABDE
 2                 3  [A, B, C, E]    ABCE
 3                 4     [A, D, E]     ADE
 4                 5     [A, B, E]     ABE
 5                 

In [9]:
#Change to string format
logs_dis = ['Order_log_dis', 'Loops_log_dis', 'Skips_log_dis', 'Order_Loops_log_dis', 'Loops_Skips_log_dis', 'Skips_Order_log_dis', 'Order_Loops_Skips_log_dis']

for i in range(len(logs)):
    logs_dis[i] = Lev(logs[i])

0
0
0
0
0
0
0


In [10]:
#Average Levenshtein distance
for i in range(len(logs_dis)):
    variety = np.average(logs_dis[i])
    print(str(i)+':', variety)

0: 1.6666666666666667
1: 1.1666666666666667
2: 1.1666666666666667
3: 2.0
4: 2.25
5: 1.7083333333333333
6: 2.2037037037037037


# Variety

In [11]:
#Calculate linkage Matrix
from scipy.cluster.hierarchy import linkage

#Distances after clustering
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import squareform

#For Evaluation
from scipy.integrate import simpson
from numpy import trapz
import scipy.stats
from scipy.cluster.hierarchy import fcluster

In [12]:
#Change to string format
logs_link = ['Order_log_link', 'Loops_log_link', 'Skips_log_link', 'Order_Loops_log_link', 'Loops_Skips_log_link', 'Skips_Order_log_link', 'Order_Loops_Skips_log_link']

for i in range(len(logs_dis)):
    #logs_link[i] = linkage(logs_dis[i], "complete")
    logs_link[i] = linkage(squareform(logs_dis[i]), "centroid")

In [17]:
logs_link

[array([[0.        , 1.        , 2.        , 2.        ],
        [2.        , 6.        , 1.73205081, 3.        ],
        [3.        , 7.        , 1.63299316, 4.        ],
        [4.        , 8.        , 1.58113883, 5.        ],
        [5.        , 9.        , 1.54919334, 6.        ]]),
 array([[0.        , 1.        , 1.        , 2.        ],
        [3.        , 6.        , 0.8660254 , 3.        ],
        [4.        , 5.        , 1.        , 2.        ],
        [7.        , 8.        , 1.38443731, 5.        ],
        [2.        , 9.        , 1.42828569, 6.        ]]),
 array([[0.        , 1.        , 1.        , 2.        ],
        [3.        , 6.        , 0.8660254 , 3.        ],
        [4.        , 5.        , 1.        , 2.        ],
        [7.        , 8.        , 1.38443731, 5.        ],
        [2.        , 9.        , 1.42828569, 6.        ]]),
 array([[ 0.        ,  9.        ,  1.        ,  2.        ],
        [10.        , 12.        ,  0.8660254 ,  3.        ],


In [13]:
#Calculate MST --> Variety

def mst(linkage_matrix):
    x = 0
    for i in linkage_matrix:
        x += i[2]
    return x


In [14]:
#logs = [Order_log, Loops_log, Skips_log, Order_Loops_log, Loops_Skips_log, Skips_Order_log, Order_Loops_Skips_log]

for i in range(len(logs_link)):
    variety = mst(logs_link[i])
    print(str(i)+':', variety)

0: 8.495376137991485
1: 5.678748399979353
2: 5.678748399979353
3: 15.742235373338723
4: 14.580676734260996
5: 14.171371007438497
6: 22.926439305938665
